# MLP with Feature Extractor

In [2]:
import torch
import os
import optuna
import time
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Subset, random_split, ConcatDataset, DataLoader
from PIL import Image
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from skimage.feature import local_binary_pattern

import cv2

/home/jose/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-24 22:36:09.038272: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 22:36:12.336832: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Función para Feature Extractor LBP (Local Binary Patterns)

In [3]:
def extract_lbp_features(image):
    if len(image.shape) == 3 and image.shape[2] == 3:  # Color image
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:  # Single-channel image
        gray = image

    lbp = local_binary_pattern(gray, 8, 1, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 257), range=(0, 256))
    hist = hist.astype(np.float32)
    hist /= np.sum(hist)
    return hist

## Normalización de datos, además creación de training y testing datasets

In [4]:
"""
-- Normalize pixel values and create training and testing datasets --

This cell normalizes the pixel values and creates a training
and testing datasets considering the stratify technique.
"""

parent_folder_path = 'images/cropped_dataset/'
categories = {'COVID': 0, 'Lung_Opacity': 1, 'Normal': 2, 'Viral_Pneumonia': 3}
arrays = []
category_amount = []

# get the category with the least images
for category in categories.keys():
    folder_path = os.path.join(parent_folder_path, category)
    image_files = os.listdir(folder_path)
    category_amount.append(len(image_files))

max_training = min(category_amount)

# convert the images into a pytorch dataset
for cat_folder, value in categories.items():

    folder_path = os.path.join(parent_folder_path, cat_folder)
    image_files = os.listdir(folder_path)

    for i, file_name in enumerate(image_files):

        if i >= max_training:
            break

        file_path = os.path.join(folder_path, file_name)
        image = Image.open(file_path)
        image_array = np.array(image)

        # verify all images are of the desired size
        if image.size != (250, 250):
            print(file_path, " IS NOT 250x250, it is: ", image.size)
            continue

        if image_array.shape != (250, 250):
            image_array = np.dot(image_array[..., :3], [0.2989, 0.5870, 0.1140])

        # extract LBP features from image
        image_features = extract_lbp_features(image_array)

        arrays.append(image_features)

# reshape the array
arrays = np.array(arrays).astype(np.float32)
arrays = arrays / np.max(arrays)

arrays_labels = [0] * max_training
arrays_labels += [1] * max_training
arrays_labels += [2] * max_training
arrays_labels += [3] * max_training

arrays_labels = np.array(arrays_labels)

X_train, X_test, y_train, y_test = train_test_split(arrays, arrays_labels, test_size=0.2, random_state=42, stratify=arrays_labels)

y_train = to_categorical(y_train.astype(int), num_classes=4)
y_test = to_categorical(y_test.astype(int), num_classes=4)

/home/jose/.local/lib/python3.8/site-packages/skimage/feature/texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


## Arquitectura 1 para modelo MLP

In [5]:
# Modelo de red MLP personalizado
model_mlp = Sequential()
model_mlp.add(Dense(64, activation='relu', input_shape=(256,)))

model_mlp.add(Dense(32, activation='sigmoid'))

model_mlp.add(Dense(16, activation='relu'))

model_mlp.add(Dense(4, activation='softmax'))

optimizer = Adam(learning_rate=0.001)
model_mlp.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model_mlp.fit(X_train, y_train, batch_size=128, epochs=500, verbose=1)

Epoch 1/500
72/72 [==============================] - 2s 4ms/step - loss: 1.3888 - accuracy: 0.2708
Epoch 2/500
72/72 [==============================] - 0s 4ms/step - loss: 1.3784 - accuracy: 0.3080
Epoch 3/500
72/72 [==============================] - 0s 4ms/step - loss: 1.3575 - accuracy: 0.4193
Epoch 4/500
72/72 [==============================] - 0s 4ms/step - loss: 1.3096 - accuracy: 0.4369
Epoch 5/500
72/72 [==============================] - 0s 4ms/step - loss: 1.2602 - accuracy: 0.4385
Epoch 6/500
72/72 [==============================] - 0s 4ms/step - loss: 1.2374 - accuracy: 0.4519
Epoch 7/500
72/72 [==============================] - 0s 4ms/step - loss: 1.2179 - accuracy: 0.4569
Epoch 8/500
72/72 [==============================] - 0s 4ms/step - loss: 1.1978 - accuracy: 0.4566
Epoch 9/500
72/72 [==============================] - 0s 4ms/step - loss: 1.1815 - accuracy: 0.4693
Epoch 10/500
72/72 [==============================] - 0s 4ms/step - loss: 1.1711 - accuracy: 0.4732
Epoch 11/

In [6]:
pred = model_mlp.predict(X_test)
print(pred)

72/72 [==============================] - 0s 2ms/step
[[0.09557159 0.43964577 0.29677176 0.16801083]
 [0.12742788 0.572948   0.17691833 0.12270575]
 [0.02188586 0.08467767 0.8910168  0.00241965]
 ...
 [0.05793305 0.519329   0.38936576 0.03337219]
 [0.03933241 0.00861321 0.01527215 0.9367823 ]
 [0.06936229 0.6236218  0.3056267  0.00138915]]


## Arquitectura 2 para modelo MLP

In [7]:
# Modelo de red MLP personalizado
model_mlp = Sequential()
model_mlp.add(Dense(64, activation='relu', input_shape=(256,)))

model_mlp.add(Dense(32, activation='sigmoid'))

model_mlp.add(Dense(128, activation='relu'))

model_mlp.add(Dense(4, activation='softmax'))

optimizer = Adam(learning_rate=0.001)
model_mlp.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model_mlp.fit(X_train, y_train, batch_size=128, epochs=500, verbose=1)

Epoch 1/500
72/72 [==============================] - 1s 3ms/step - loss: 1.3890 - accuracy: 0.2568
Epoch 2/500
72/72 [==============================] - 0s 3ms/step - loss: 1.3806 - accuracy: 0.2749
Epoch 3/500
72/72 [==============================] - 0s 3ms/step - loss: 1.3594 - accuracy: 0.3356
Epoch 4/500
72/72 [==============================] - 0s 3ms/step - loss: 1.3010 - accuracy: 0.3903
Epoch 5/500
72/72 [==============================] - 0s 3ms/step - loss: 1.2593 - accuracy: 0.4127
Epoch 6/500
72/72 [==============================] - 0s 3ms/step - loss: 1.2453 - accuracy: 0.4261
Epoch 7/500
72/72 [==============================] - 0s 3ms/step - loss: 1.2330 - accuracy: 0.4309
Epoch 8/500
72/72 [==============================] - 0s 3ms/step - loss: 1.2201 - accuracy: 0.4368
Epoch 9/500
72/72 [==============================] - 0s 3ms/step - loss: 1.2037 - accuracy: 0.4521
Epoch 10/500
72/72 [==============================] - 0s 4ms/step - loss: 1.1910 - accuracy: 0.4623
Epoch 11/

In [8]:
pred = model_mlp.predict(X_test)
print(pred)

72/72 [==============================] - 0s 1ms/step
[[0.07505395 0.35026368 0.18071908 0.39396316]
 [0.10142038 0.4957079  0.08536626 0.3175055 ]
 [0.01942712 0.12725851 0.8505216  0.0027928 ]
 ...
 [0.05346331 0.5797794  0.3150145  0.05174275]
 [0.03493651 0.01515577 0.02557346 0.9243343 ]
 [0.04922816 0.8411382  0.10843005 0.00120359]]


## Arquitectura 3 para modelo MLP

In [9]:
# Modelo de red MLP personalizado
model_mlp = Sequential()
model_mlp.add(Dense(64, activation='relu', input_shape=(256,)))

model_mlp.add(Dense(16, activation='relu'))

model_mlp.add(Dense(256, activation='relu'))

model_mlp.add(Dense(4, activation='softmax'))

optimizer = Adam(learning_rate=0.001)
model_mlp.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model_mlp.fit(X_train, y_train, batch_size=128, epochs=500, verbose=1)

Epoch 1/500
72/72 [==============================] - 1s 3ms/step - loss: 1.3667 - accuracy: 0.3377
Epoch 2/500
72/72 [==============================] - 0s 3ms/step - loss: 1.2591 - accuracy: 0.4278
Epoch 3/500
72/72 [==============================] - 0s 4ms/step - loss: 1.1777 - accuracy: 0.4654
Epoch 4/500
72/72 [==============================] - 0s 3ms/step - loss: 1.1241 - accuracy: 0.5066
Epoch 5/500
72/72 [==============================] - 0s 3ms/step - loss: 1.0748 - accuracy: 0.5283
Epoch 6/500
72/72 [==============================] - 0s 3ms/step - loss: 1.0487 - accuracy: 0.5365
Epoch 7/500
72/72 [==============================] - 0s 4ms/step - loss: 1.0280 - accuracy: 0.5467
Epoch 8/500
72/72 [==============================] - 0s 3ms/step - loss: 1.0103 - accuracy: 0.5540
Epoch 9/500
72/72 [==============================] - 0s 3ms/step - loss: 0.9914 - accuracy: 0.5648
Epoch 10/500
72/72 [==============================] - 0s 4ms/step - loss: 0.9794 - accuracy: 0.5755
Epoch 11/

In [10]:
pred = model_mlp.predict(X_test)
print(pred)

72/72 [==============================] - 0s 3ms/step
[[2.40295157e-02 6.18923187e-01 3.47389251e-01 9.65801533e-03]
 [1.34922072e-01 6.65022194e-01 1.21185884e-01 7.88698345e-02]
 [2.80336873e-03 4.85583916e-02 9.48604465e-01 3.37479723e-05]
 ...
 [5.25821261e-02 6.36176765e-01 3.08633715e-01 2.60746898e-03]
 [9.61818844e-02 6.00523409e-03 2.32178215e-02 8.74595106e-01]
 [9.59137455e-02 8.41882050e-01 6.19970299e-02 2.07202233e-04]]
